In [108]:
import pandas as pd
import numpy as np

# Next steps:

several questions:

- How to choose the loss function?
- How to interpret the loss function?
- Predict with more than two classes
- 

* Make predictions
https://machinelearningmastery.com/how-to-make-classification-and-regression-predictions-for-deep-learning-models-in-keras/

https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/

https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

https://stackoverflow.com/questions/59260861/why-is-my-loss-function-return-negative-values#:~:text=1%20Answer&text=The%20loss%20is%20just%20a,per%20the%20reference%20code%20here.

* Loss functions
https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-class

# Load text data

In [109]:
path = '/Users/liviaclarete/coding/nlp-okeyso/data/01_clean_data.csv'
df = pd.read_csv(path)
print(df.shape)
df.head()

(8292, 6)


,name,question_id,body,users,messages,date
0,Ask a College Student,8286,Hello\n,1,1,2020-02-06 23:14:50
1,Ask a College Student,8259,"Hi, I haven’t been to a college because I move...",2,2,2020-02-01 17:46:54
2,Ask a College Student,8219,I think I'm addicted to nicotine but I can go ...,2,2,2020-01-22 20:56:18
3,Ask a College Student,8218,I think I'm addicted to nicotine but I can go ...,1,1,2020-01-22 20:56:13
4,Ask a College Student,8217,I think I'm addicted to nicotine but I can go ...,2,6,2020-01-22 20:56:13


In [3]:
# checking the categories
categs = df.name.value_counts().reset_index()
categs.columns = ['themas', 'name']
categs

,themas,name
0,Dating & Relationships,2254
1,Sex,1760
2,Stress and self-care,1233
3,Birth Control/Pregnancy,698
4,Body/Anatomy,696
5,Identity,657
6,Ask a College Student,619
7,STIs/Safer Sex,375


# Preprocessing

## Encode the labels
In this case of supervised task, transform the labels into codes

In [27]:
# encode the categories
codes = categs.to_dict()['themas']

# invert the position of key and value
codes_dict = dict({(v,k) for k,v in codes.items()})

# replace the code 0 by 8
# accodingly to stackoverflow users, it can confuse the classifier
# https://stackoverflow.com/questions/2213334/in-python-i-have-a-dictionary-how-do-i-change-the-keys-of-this-dictionary/7666742
codes_dict['Dating & Relationships'] = 8

# create a new category within the df
df['categs'] = df.name.replace(codes_dict)

# create corpus and labels
corpus = df.body.to_list()

# create a list and convert it to a numpy array
labels = df.categs.to_list()

# https://stackoverflow.com/questions/58682026/failed-to-find-data-adapter-that-can-handle-input-class-numpy-ndarray-cl
labels = np.uint8(labels)

## Encode the text

In [28]:
# Encode my text
from word_embedding_preprocessing import padding_encoded_vector_func

padded_sent = padding_encoded_vector_func(corpus, extra_size=5)
padded_sent

array([[ 4249,     0,     0, ...,     0,     0,     0],
       [ 5885, 10381, 11906, ...,     0,     0,     0],
       [10381,  3742,  8753, ...,     0,     0,     0],
       ...,
       [10761,  7969,   720, ...,     0,     0,     0],
       [ 3340,  1785,  3956, ...,     0,     0,     0],
       [ 6783,  8044,  6394, ...,     0,     0,     0]], dtype=int32)

In [70]:
padded_sent.shape

(8292, 268)

In [74]:
## Split data into train and test set, and validation set
X = padded_sent[:8000]
y = labels[:8000]

# test over the predict_classes model's method
X_val = padded_sent[8000:]
y_val = labels[8000:]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [77]:
padded_sent.shape

(8292, 268)

In [78]:
X.shape

(8000, 268)

In [79]:
X_val.shape

(292, 268)

## Return vocab size and longest sent

In [80]:
# find the vocab size
from word_embedding_preprocessing import vocab_size_func, longest_sentence_func

vocab_size = vocab_size_func(corpus) + 5
print(vocab_size)

longest_sent = longest_sentence_func(corpus)
print(longest_sent)

12140
('So I’ve been dealing with undiagnosed depression for a few years now. Early 2018 was really bad, but over the summer I was 100% fine. Soon after school started back up again, I got worse. I’m not as bad as I was but I’m really not good. I think I had my first anxiety attack tonight (I’m not sure though) I’m concidering getting help but I also really don’t want to. I’m scared. I don’t want to tell my mom but I know I’d have to (I’m only 17) I honestly don’t know what to do. Getting help would be expensive and I don’t want to put my mom through that. I also don’t want to talk to my mom about it all and I know she’d ask questions. At the same time I want her to know because she’s always yelling at me for my grades and sleeping too much and stuff but maybe if she knew she’d understand (she’s had anxiety and depression but she somehow has noticed the signs but hasn’t realized what they mean?) I guess I’m asking if it’s worth it to try and get help. (Even tho I don’t really want to b

## Load pre-trained models
In this case, using GloVe. It will be used as a weigth within the model

In [7]:
# load pre-trained model: it will be used as a weight
from encode_glove import load_glove_file, load_embedding_matrix
path_glove = '/Users/liviaclarete/Downloads/glove.6B/glove.6B.100d.txt'


In [9]:
embedding = load_embedding_matrix(corpus, vocab_size, path_glove)

In [16]:
embedding.shape

(12140, 100)

# Modeling data

## Create a model

In [ ]:
# 3) Modeling data
from word_embedding_model import create_sequencial_model, create_functional_model

In [81]:
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Flatten
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam


def create_sequencial_model(list_of_strings, vocab_size, learn_rate, embedding_matrix):

    # create a model
    model = Sequential()

    # ADD LAYERS TO THE MODEL
    
    # word embeddings inputs
    # vocab_size = vocab_size_func(list_of_strings)
    _, len_long_sentence =  longest_sentence_func(list_of_strings)
    
    # add the Embedding layer
    model.add(Embedding(
        # size of the vocabulary
        ## or the total number of unique words in a corpus
        vocab_size, 
        # number of dimensions for each word vector to be created
        # in this case, the shape of the embedding_matrix
        # not sure why, but it worked
        embedding_matrix.shape[1], 
        # len of the longest sentence
        input_length=len_long_sentence,
        weights=[embedding_matrix]
        ))

    # flatten the layer so it can be used directly with 
    # the densely connected layer
    # as we can see in the summary, the flatten array has a shape of 140
    # instead of (7, 20)
    # the flatten array is directly connected to the Dense layer
    model.add(Flatten())

    # add a Dense layer
    model.add(Dense(
        # https://keras.io/guides/sequential_model/
        # Not sure aht this 1 is. The author call it 'neuran'
        # maybe it is a 'neuron' as 'neuran' looks like a typo
        # but it impacts on the number of trainable parameters
        1,
        # as it is a binary classification, 
        # we use sigmoid as the loss fucntion
        activation='sigmoid'))
    

    # COMPILE THE MODEL
    adam = Adam(lr=learn_rate)
    
    model.compile(
        optimizer=adam,
        # loss function for binary classification
        loss='categorical_crossentropy',
        metrics=['acc']
    )
    
    return model

In [82]:

learn_rate = 0.001
model = create_sequencial_model(corpus, vocab_size, learn_rate, embedding)

In [83]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 268, 100)          1214000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 26800)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 26801     
Total params: 1,240,801
Trainable params: 1,240,801
Non-trainable params: 0
_________________________________________________________________


## Fit the model

In [85]:
# train the model using X_train and y_train

epochs=300
batch_size=1

model.fit(
    # encoded matrix
    X_train,
    y_train,
    batch_size=batch_size,
    verbose=1
)

6000/6000 [==============================] - 120s 20ms/step - loss: 5.3837e-07 - acc: 0.0975


## Evaluate the model

In [86]:
# 4) evaluate the model
loss, accuracy = model.evaluate(
    # padded array for the test set
    X_test, 
    # labels for the test set
    y_test, 
    verbose=1)

print('Loss:', round(loss))

# In the output, that model accuracy is 1.00 i.e. 100 percent.
print('Accuracy: ', accuracy)

63/63 [==============================] - 0s 3ms/step - loss: 5.4115e-07 - acc: 0.0970
Loss: 0
Accuracy:  0.09700000286102295


## Predict

In [87]:
X_val

array([[11438,  8475,  8044, ...,     0,     0,     0],
       [  108,  8375,  4326, ...,     0,     0,     0],
       [ 5854, 10381,  2142, ...,     0,     0,     0],
       ...,
       [10761,  7969,   720, ...,     0,     0,     0],
       [ 3340,  1785,  3956, ...,     0,     0,     0],
       [ 6783,  8044,  6394, ...,     0,     0,     0]], dtype=int32)

In [90]:
# use the X_val to predict the results 
# and compare the output with the y_val

# the x_new should have the same size of the initial arrays
X_new = X_val

In [92]:
# predict the classes
pred = model.predict_classes(X_new)

In [107]:
# returning only two categs, instead of 8!
pd.Series(sum(pred.tolist(), [])).value_counts()

0    159
1    133
dtype: int64